<div style="background-color: #1B1A21; text-align: right; margin-bottom: -1px">
    <img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-banner.png" style="padding: 0px; padding-right: 20px; margin: 0px; padding-top: 20px; height: 60px"/>
    <img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/banner-colors.png" style="width:100%; height: 50px; padding: 0px; margin: 0px; margin-bottom: -8px"/>
</div>

In [ ]:
!pip install langchain --quiet
!pip install openai --quiet
!pip install singlestoredb --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet

In [ ]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("http://leavcom.com/pdf/DBpdf.pdf")

data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import SingleStoreDB

os.environ["SINGLESTOREDB_URL"] = "admin:<password>@<host>:3306/pdf_db"

docsearch = SingleStoreDB.from_documents(
    texts,
    embedder,
    table_name = "pdf_docs2",
)

In [ ]:
%%sql

USE pdf_db;
SELECT JSON_ARRAY_UNPACK_F32(vector)
FROM pdf_docs2
LIMIT 1;

In [ ]:
query_text = "Will object-oriented databases be commercially successful?"

docs = docsearch.similarity_search(query_text)

print(docs[0].page_content)

In [ ]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])